In [ ]:
# Importing necessary libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, KFold
import torch
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

# Create a directory for our analysis results
os.makedirs("ensemble_analysis", exist_ok=True)
os.makedirs("ensemble_analysis/figures", exist_ok=True)

In [ ]:
# Function to load XGBoost and Transformer predictions
def load_predictions():
    # Load Transformer predictions
    transformer_preds = pd.read_csv("transformers_logs/results/test_predictions.csv")
    
    # Load XGBoost predictions
    # Assuming the XGBoost predictions are stored in a similar format
    # If format is different, this will need to be modified
    try:
        xgb_preds = pd.read_csv("results/test_predictions.csv")
    except FileNotFoundError:
        # Try alternative formats that might exist
        try:
            # Look for any JSON files in results directory
            json_files = [f for f in os.listdir("results") if f.endswith(".json") and "metrics" in f]
            if json_files:
                # Sort by date (assuming naming includes date)
                json_files.sort(reverse=True)
                # Load the most recent metrics file
                with open(f"results/{json_files[0]}", "r") as f:
                    xgb_metrics = json.load(f)
                
                # For now, use transformer predictions structure but will replace with actual XGB predictions
                print(f"Found XGBoost metrics but no predictions file. Will need to generate predictions.")
                xgb_preds = pd.read_csv("split_data/test_data.csv")
            else:
                raise FileNotFoundError("No XGBoost prediction or metrics files found")
        except Exception as e:
            print(f"Error loading XGBoost predictions: {e}")
            print("Will attempt to load and run XGBoost model to generate predictions.")
            # Create a placeholder with transformer structure for now
            xgb_preds = transformer_preds[['header', 'position', 'target']].copy()
    
    return transformer_preds, xgb_preds

# Load model predictions
transformer_preds, xgb_preds = load_predictions()

# Display the first few rows of each prediction dataframe
print("Transformer Predictions:")
display(transformer_preds.head())

print("\nXGBoost Predictions or Test Data:")
display(xgb_preds.head())

# Check for column name inconsistencies
print("\nTransformer columns:", transformer_preds.columns.tolist())
print("XGBoost columns:", xgb_preds.columns.tolist())

# Check if we need to identify overlap between the two sets
transformer_id_cols = [col for col in transformer_preds.columns if col.lower() in ['header', 'uniprot id', 'position', 'pos']]
xgb_id_cols = [col for col in xgb_preds.columns if col.lower() in ['header', 'uniprot id', 'position', 'pos']]

print("\nTransformer ID columns:", transformer_id_cols)
print("XGBoost ID columns:", xgb_id_cols)

# Try to determine common identifier columns 
header_col_transformer = [col for col in transformer_id_cols if 'header' in col.lower() or 'id' in col.lower()]
header_col_xgb = [col for col in xgb_id_cols if 'header' in col.lower() or 'id' in col.lower()]
pos_col_transformer = [col for col in transformer_id_cols if 'pos' in col.lower()]
pos_col_xgb = [col for col in xgb_id_cols if 'pos' in col.lower()]

if header_col_transformer and header_col_xgb and pos_col_transformer and pos_col_xgb:
    header_col_transformer = header_col_transformer[0]
    header_col_xgb = header_col_xgb[0]
    pos_col_transformer = pos_col_transformer[0]
    pos_col_xgb = pos_col_xgb[0]
    
    # Check overlap between datasets
    transformer_ids = set(zip(transformer_preds[header_col_transformer], transformer_preds[pos_col_transformer]))
    xgb_ids = set(zip(xgb_preds[header_col_xgb], xgb_preds[pos_col_xgb]))
    
    overlap = transformer_ids.intersection(xgb_ids)
    
    print(f"\nTransformer dataset has {len(transformer_ids)} unique samples")
    print(f"XGBoost dataset has {len(xgb_ids)} unique samples")
    print(f"Overlap: {len(overlap)} samples ({len(overlap)/len(transformer_ids)*100:.2f}% of transformer, {len(overlap)/len(xgb_ids)*100:.2f}% of XGBoost)")
else:
    print("\nCould not automatically determine ID columns. Please check the dataframes.")

In [ ]:
# Function to load and prepare the XGBoost model if needed
def load_xgboost_model_and_predict():
    print("Loading XGBoost model and making predictions...")
    
    try:
        # Try to load the model
        xgb_model = xgb.Booster()
        xgb_model.load_model("results/phosphorylation_xgb_model.json")
        
        # Load test data
        test_data = pd.read_csv("split_data/test_data.csv")
        
        # Prepare data for prediction
        target_col = 'target'
        id_cols = ['Header', 'Position']
        
        # Identify feature columns (anything not in id_cols or target_col)
        feature_cols = [col for col in test_data.columns 
                        if col not in id_cols + [target_col]]
        
        if not feature_cols:
            print("No feature columns found in test data!")
            return None
            
        X_test = test_data[feature_cols]
        
        # Convert to DMatrix
        dtest = xgb.DMatrix(X_test)
        
        # Make predictions
        xgb_probs = xgb_model.predict(dtest)
        
        # Create prediction dataframe
        xgb_preds = pd.DataFrame({
            'Header': test_data['Header'],
            'Position': test_data['Position'],
            'target': test_data['target'],
            'probability': xgb_probs,
            'prediction': (xgb_probs > 0.5).astype(int)
        })
        
        print("XGBoost predictions generated successfully")
        return xgb_preds
        
    except Exception as e:
        print(f"Error loading or running XGBoost model: {e}")
        return None

# Function to merge predictions, ensuring only common samples are used
def merge_predictions(transformer_preds, xgb_preds):
    # Identify key columns for merging
    transformer_cols = transformer_preds.columns
    xgb_cols = xgb_preds.columns
    
    # Try to find ID columns automatically
    header_col_transformer = [col for col in transformer_cols if 'header' in col.lower() or 'id' in col.lower()]
    pos_col_transformer = [col for col in transformer_cols if 'pos' in col.lower()]
    header_col_xgb = [col for col in xgb_cols if 'header' in col.lower() or 'id' in col.lower()]
    pos_col_xgb = [col for col in xgb_cols if 'pos' in col.lower()]
    
    # Check if we found the columns
    if (not header_col_transformer or not pos_col_transformer or 
        not header_col_xgb or not pos_col_xgb):
        print("Could not automatically identify ID columns. Using default column names.")
        # Use default column names
        header_col_transformer = 'header'
        pos_col_transformer = 'position'
        header_col_xgb = 'Header'
        pos_col_xgb = 'Position'
    else:
        header_col_transformer = header_col_transformer[0]
        pos_col_transformer = pos_col_transformer[0]
        header_col_xgb = header_col_xgb[0]
        pos_col_xgb = pos_col_xgb[0]
    
    print(f"Merging on {header_col_transformer}/{pos_col_transformer} from transformer and {header_col_xgb}/{pos_col_xgb} from XGBoost")
    
    # Check if each dataframe has the necessary columns
    for df, name, header_col, pos_col in [
        (transformer_preds, "Transformer", header_col_transformer, pos_col_transformer),
        (xgb_preds, "XGBoost", header_col_xgb, pos_col_xgb)
    ]:
        if header_col not in df.columns:
            print(f"Error: {header_col} not found in {name} predictions")
            return None
        if pos_col not in df.columns:
            print(f"Error: {pos_col} not found in {name} predictions")
            return None
    
    # Perform the merge
    merged_preds = pd.merge(
        transformer_preds,
        xgb_preds,
        left_on=[header_col_transformer, pos_col_transformer],
        right_on=[header_col_xgb, pos_col_xgb],
        suffixes=('_transformer', '_xgb')
    )
    
    print(f"Successfully merged {len(merged_preds)} samples")
    
    # Check for target column inconsistencies
    target_col_transformer = [col for col in transformer_cols if 'target' in col.lower()]
    target_col_xgb = [col for col in xgb_cols if 'target' in col.lower()]
    
    if target_col_transformer and target_col_xgb:
        target_col_transformer = target_col_transformer[0]
        target_col_xgb = target_col_xgb[0]
        
        # Check if targets match
        if target_col_transformer in merged_preds and target_col_xgb in merged_preds:
            targets_match = (merged_preds[target_col_transformer] == merged_preds[target_col_xgb]).all()
            if not targets_match:
                print("Warning: Target values differ between the two prediction sets!")
            else:
                print("Target values are consistent between the datasets")
    
    return merged_preds

# Check if XGBoost predictions need to be generated
if ('probability' not in xgb_preds.columns or 'prediction' not in xgb_preds.columns):
    print("XGBoost predictions need to be generated")
    generated_preds = load_xgboost_model_and_predict()
    if generated_preds is not None:
        xgb_preds = generated_preds

# Merge predictions
merged_preds = merge_predictions(transformer_preds, xgb_preds)

if merged_preds is not None:
    # Display the merged dataframe
    print("\nMerged predictions:")
    display(merged_preds.head())
    print(f"Shape: {merged_preds.shape}")
    
    # Save merged predictions for later use
    merged_preds.to_csv("ensemble_analysis/merged_predictions.csv", index=False)
else:
    print("Failed to merge prediction datasets. Please check the data and columns.")

In [ ]:
# If we successfully merged the predictions, proceed with ensemble methods
if merged_preds is not None:
    # Identify target and prediction columns
    # Try to find them automatically
    target_cols = [col for col in merged_preds.columns if 'target' in col.lower()]
    transformer_prob_cols = [col for col in merged_preds.columns 
                            if 'prob' in col.lower() and 'transformer' in col.lower()]
    xgb_prob_cols = [col for col in merged_preds.columns 
                    if 'prob' in col.lower() and 'xgb' in col.lower()]
    
    if target_cols:
        target_col = target_cols[0]
    else:
        target_col = 'target'  # default
        
    if transformer_prob_cols:
        transformer_prob_col = transformer_prob_cols[0]
    else:
        transformer_prob_col = 'probability_transformer'  # default
        
    if xgb_prob_cols:
        xgb_prob_col = xgb_prob_cols[0]
    else:
        xgb_prob_col = 'probability_xgb'  # default
    
    print(f"Using {target_col} as target column")
    print(f"Using {transformer_prob_col} as transformer probability column")
    print(f"Using {xgb_prob_col} as XGBoost probability column")
    
    # Create prediction columns if they don't exist
    if 'prediction_transformer' not in merged_preds.columns:
        merged_preds['prediction_transformer'] = (merged_preds[transformer_prob_col] > 0.5).astype(int)
    
    if 'prediction_xgb' not in merged_preds.columns:
        merged_preds['prediction_xgb'] = (merged_preds[xgb_prob_col] > 0.5).astype(int)
    
    # Function to evaluate model performance
    def evaluate_model(y_true, y_pred, y_prob, model_name):
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        roc_auc = roc_auc_score(y_true, y_prob)
        
        metrics = {
            'Model': model_name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'AUC': roc_auc
        }
        
        return metrics
    
    # Evaluate both models
    metrics_transformer = evaluate_model(
        merged_preds[target_col], 
        merged_preds['prediction_transformer'], 
        merged_preds[transformer_prob_col],
        'Transformer'
    )
    
    metrics_xgb = evaluate_model(
        merged_preds[target_col], 
        merged_preds['prediction_xgb'], 
        merged_preds[xgb_prob_col],
        'XGBoost'
    )
    
    # Combine metrics into a dataframe
    metrics_df = pd.DataFrame([metrics_transformer, metrics_xgb])
    print("Performance metrics:")
    display(metrics_df)
    
    # Save metrics
    metrics_df.to_csv("ensemble_analysis/base_model_metrics.csv", index=False)
    
    # Now we can proceed with analysis and ensemble methods
    print("Ready to proceed with analysis and ensemble methods")
else:
    print("Cannot proceed with analysis as predictions could not be merged")

In [ ]:
# Function for plotting confusion matrices
def plot_confusion_matrix(y_true, y_pred, title, ax):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(
        cm, 
        annot=True, 
        fmt="d", 
        cmap="Blues", 
        cbar=False,
        ax=ax
    )
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title(title)
    return cm

# Create confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

cm_transformer = plot_confusion_matrix(
    merged_preds[target_col], 
    merged_preds['prediction_transformer'], 
    'Transformer Confusion Matrix', 
    axes[0]
)

cm_xgb = plot_confusion_matrix(
    merged_preds[target_col], 
    merged_preds['prediction_xgb'], 
    'XGBoost Confusion Matrix', 
    axes[1]
)

plt.tight_layout()
plt.savefig('ensemble_analysis/figures/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

# Create a matrix showing model agreement and disagreement
agreement_df = pd.DataFrame({
    'XGBoost': merged_preds['prediction_xgb'],
    'Transformer': merged_preds['prediction_transformer'],
    'True': merged_preds[target_col]
})

# Count occurrences of each combination
agreement_counts = agreement_df.groupby(['XGBoost', 'Transformer', 'True']).size().reset_index(name='Count')
print("\nModel agreement and disagreement:")
display(agreement_counts)

# Plot model agreement
plt.figure(figsize=(10, 8))
agreement_pivot = pd.pivot_table(
    agreement_counts,
    index=['XGBoost', 'Transformer'],
    columns='True',
    values='Count',
    fill_value=0
)

# Plot as heatmap
sns.heatmap(agreement_pivot, annot=True, fmt="d", cmap="viridis")
plt.title('Model Agreement vs. True Labels')
plt.xlabel('True Label')
plt.ylabel('Predictions (XGBoost, Transformer)')

plt.tight_layout()
plt.savefig('ensemble_analysis/figures/model_agreement.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Find where both models fail
both_fail = merged_preds[
    ((merged_preds['prediction_transformer'] != merged_preds[target_col]) & 
     (merged_preds['prediction_xgb'] != merged_preds[target_col]))
]

# Find where both models succeed
both_succeed = merged_preds[
    ((merged_preds['prediction_transformer'] == merged_preds[target_col]) & 
     (merged_preds['prediction_xgb'] == merged_preds[target_col]))
]

# Find where only transformer succeeds
only_transformer_succeeds = merged_preds[
    ((merged_preds['prediction_transformer'] == merged_preds[target_col]) & 
     (merged_preds['prediction_xgb'] != merged_preds[target_col]))
]

# Find where only XGBoost succeeds
only_xgb_succeeds = merged_preds[
    ((merged_preds['prediction_transformer'] != merged_preds[target_col]) & 
     (merged_preds['prediction_xgb'] == merged_preds[target_col]))
]

print(f"\nBoth models fail: {len(both_fail)} samples ({len(both_fail)/len(merged_preds)*100:.2f}%)")
print(f"Both models succeed: {len(both_succeed)} samples ({len(both_succeed)/len(merged_preds)*100:.2f}%)")
print(f"Only Transformer succeeds: {len(only_transformer_succeeds)} samples ({len(only_transformer_succeeds)/len(merged_preds)*100:.2f}%)")
print(f"Only XGBoost succeeds: {len(only_xgb_succeeds)} samples ({len(only_xgb_succeeds)/len(merged_preds)*100:.2f}%)")

# Save these results
result_sets = {
    'both_fail': both_fail,
    'both_succeed': both_succeed,
    'only_transformer_succeeds': only_transformer_succeeds,
    'only_xgb_succeeds': only_xgb_succeeds
}

for name, df in result_sets.items():
    df.to_csv(f"ensemble_analysis/{name}.csv", index=False)

In [ ]:
# Analyzing sequence patterns in failures
def analyze_sequence_patterns(dataframe, title):
    # Check if 'sequence' column exists
    sequence_cols = [col for col in dataframe.columns if 'sequence' in col.lower()]
    if not sequence_cols:
        print(f"No sequence data available in {title}")
        return None
        
    sequence_col = sequence_cols[0]
    
    # Extract center amino acid from each sequence
    try:
        # Try to extract center position (assuming sequence is of odd length)
        dataframe['center_aa'] = dataframe[sequence_col].apply(
            lambda seq: seq[len(seq)//2] if len(seq) % 2 == 1 else seq[len(seq)//2 - 1]
        )
        
        # Count center amino acids
        center_aa_counts = dataframe['center_aa'].value_counts().reset_index()
        center_aa_counts.columns = ['Amino Acid', 'Count']
        center_aa_counts['Percentage'] = center_aa_counts['Count'] / len(dataframe) * 100
        
        print(f"\nCenter amino acid distribution in {title}:")
        display(center_aa_counts)
        
        # Plot center amino acid distribution
        plt.figure(figsize=(12, 6))
        ax = sns.barplot(x='Amino Acid', y='Count', data=center_aa_counts)
        ax.set_title(f'Center Amino Acid Distribution - {title}')
        ax.set_xlabel('Amino Acid')
        ax.set_ylabel('Count')
        
        # Add percentage labels
        for i, p in enumerate(ax.patches):
            height = p.get_height()
            ax.text(p.get_x() + p.get_width()/2.,
                    height + 0.5,
                    f'{center_aa_counts.iloc[i]["Percentage"]:.1f}%',
                    ha="center")
        
        plt.tight_layout()
        plt.savefig(f'ensemble_analysis/figures/{title.lower().replace(" ", "_")}_center_aa.png', 
                    dpi=300, bbox_inches='tight')
        plt.show()
        
        return center_aa_counts
    except Exception as e:
        print(f"Error analyzing sequences in {title}: {e}")
        return None

# Analyze amino acid patterns where both models fail or succeed
if 'sequence' in merged_preds.columns or any('sequence' in col.lower() for col in merged_preds.columns):
    both_fail_aa = analyze_sequence_patterns(both_fail, "Both Models Fail")
    both_succeed_aa = analyze_sequence_patterns(both_succeed, "Both Models Succeed")
    only_transformer_aa = analyze_sequence_patterns(only_transformer_succeeds, "Only Transformer Succeeds")
    only_xgb_aa = analyze_sequence_patterns(only_xgb_succeeds, "Only XGBoost Succeeds")
else:
    print("No sequence data available for pattern analysis")

In [ ]:
# Calculate correlation between model predictions
corr_predictions = merged_preds[['prediction_transformer', 'prediction_xgb', target_col]].corr()
corr_probabilities = merged_preds[[transformer_prob_col, xgb_prob_col, target_col]].corr()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot prediction correlation
sns.heatmap(corr_predictions, annot=True, cmap='coolwarm', vmin=-1, vmax=1, ax=axes[0])
axes[0].set_title('Correlation: Predicted Classes')

# Plot probability correlation
sns.heatmap(corr_probabilities, annot=True, cmap='coolwarm', vmin=-1, vmax=1, ax=axes[1])
axes[1].set_title('Correlation: Prediction Probabilities')

plt.tight_layout()
plt.savefig('ensemble_analysis/figures/prediction_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 1. Simple Stacking Ensemble
from sklearn.model_selection import train_test_split

# Prepare data for stacking
X_stack = merged_preds[[transformer_prob_col, xgb_prob_col]]
y_stack = merged_preds[target_col]

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_stack, y_stack, test_size=0.3, random_state=42, stratify=y_stack
)

# Train a logistic regression meta-classifier
lr_meta = LogisticRegression(random_state=42)
lr_meta.fit(X_train, y_train)

# Make predictions
y_val_pred = lr_meta.predict(X_val)
y_val_prob = lr_meta.predict_proba(X_val)[:, 1]

# Evaluate
metrics_lr_meta = evaluate_model(y_val, y_val_pred, y_val_prob, 'Stacking Logistic Regression')

# Train a small MLP meta-classifier
mlp_meta = MLPClassifier(
    hidden_layer_sizes=(10,),
    max_iter=1000,
    alpha=0.01,
    solver='adam',
    random_state=42
)
mlp_meta.fit(X_train, y_train)

# Make predictions
y_val_pred_mlp = mlp_meta.predict(X_val)
y_val_prob_mlp = mlp_meta.predict_proba(X_val)[:, 1]

# Evaluate
metrics_mlp_meta = evaluate_model(y_val, y_val_pred_mlp, y_val_prob_mlp, 'Stacking MLP')

# Combine metrics into a dataframe
stacking_metrics_df = pd.DataFrame([metrics_lr_meta, metrics_mlp_meta])
print("\nStacking ensemble performance on validation set:")
display(stacking_metrics_df)

# Apply to full dataset for comparison
y_pred_lr = lr_meta.predict(X_stack)
y_prob_lr = lr_meta.predict_proba(X_stack)[:, 1]
metrics_lr_full = evaluate_model(y_stack, y_pred_lr, y_prob_lr, 'Stacking LR (Full)')

y_pred_mlp = mlp_meta.predict(X_stack)
y_prob_mlp = mlp_meta.predict_proba(X_stack)[:, 1]
metrics_mlp_full = evaluate_model(y_stack, y_pred_mlp, y_prob_mlp, 'Stacking MLP (Full)')

# Update metrics dataframe with results on the full dataset
all_metrics_df = pd.DataFrame([
    metrics_transformer, 
    metrics_xgb, 
    metrics_lr_full, 
    metrics_mlp_full
])
print("\nAll models performance on full dataset:")
display(all_metrics_df)

# Save all metrics
all_metrics_df.to_csv("ensemble_analysis/stacking_ensemble_metrics.csv", index=False)

In [ ]:
# 2. Weighted Ensemble
from scipy.optimize import minimize

# Function to create weighted predictions
def weighted_ensemble(probs1, probs2, weight):
    """
    Combine probabilities using a weighted average.
    weight is the weight for probs1 (1-weight for probs2)
    """
    return weight * probs1 + (1 - weight) * probs2

# Function to optimize for accuracy
def neg_accuracy_score(weight, probs1, probs2, true_labels):
    """
    Returns negative accuracy (for minimization) of weighted ensemble.
    """
    weighted_probs = weighted_ensemble(probs1, probs2, weight)
    pred = (weighted_probs > 0.5).astype(int)
    return -accuracy_score(true_labels, pred)

# Function to optimize for F1 score
def neg_f1_score(weight, probs1, probs2, true_labels):
    """
    Returns negative F1 score (for minimization) of weighted ensemble.
    """
    weighted_probs = weighted_ensemble(probs1, probs2, weight)
    pred = (weighted_probs > 0.5).astype(int)
    return -f1_score(true_labels, pred)

# Find optimal weight using cross-validation
def find_optimal_weight(X1, X2, y, metric='accuracy', cv=5):
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    best_weights = []
    
    for train_idx, val_idx in kf.split(X1):
        # Split data
        X1_train, X1_val = X1[train_idx], X1[val_idx]
        X2_train, X2_val = X2[train_idx], X2[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Find best weight for this fold
        if metric == 'accuracy':
            result = minimize(
                neg_accuracy_score, 
                0.5,  # initial guess
                args=(X1_val, X2_val, y_val), 
                bounds=[(0, 1)]
            )
        elif metric == 'f1':
            result = minimize(
                neg_f1_score, 
                0.5,  # initial guess
                args=(X1_val, X2_val, y_val), 
                bounds=[(0, 1)]
            )
        
        best_weights.append(result['x'][0])
    
    # Return average of best weights from all folds
    return np.mean(best_weights)

# Convert to numpy arrays
transformer_probs = merged_preds[transformer_prob_col].values
xgb_probs = merged_preds[xgb_prob_col].values
true_labels = merged_preds[target_col].values

# Find optimal weights
weight_acc = find_optimal_weight(transformer_probs, xgb_probs, true_labels, 'accuracy')
weight_f1 = find_optimal_weight(transformer_probs, xgb_probs, true_labels, 'f1')

print(f"\nOptimal weight (accuracy) for Transformer: {weight_acc:.4f}, XGBoost: {1-weight_acc:.4f}")
print(f"Optimal weight (F1 score) for Transformer: {weight_f1:.4f}, XGBoost: {1-weight_f1:.4f}")

# Create weighted ensemble predictions
weighted_probs_acc = weighted_ensemble(transformer_probs, xgb_probs, weight_acc)
weighted_preds_acc = (weighted_probs_acc > 0.5).astype(int)

weighted_probs_f1 = weighted_ensemble(transformer_probs, xgb_probs, weight_f1)
weighted_preds_f1 = (weighted_probs_f1 > 0.5).astype(int)

# Evaluate weighted ensembles
metrics_weighted_acc = evaluate_model(
    true_labels, weighted_preds_acc, weighted_probs_acc, 'Weighted Ensemble (Accuracy)'
)
metrics_weighted_f1 = evaluate_model(
    true_labels, weighted_preds_f1, weighted_probs_f1, 'Weighted Ensemble (F1)'
)

# Add results to metrics dataframe
all_metrics_df = pd.concat([
    all_metrics_df,
    pd.DataFrame([metrics_weighted_acc, metrics_weighted_f1])
])

print("\nAll models performance including weighted ensembles:")
display(all_metrics_df)

# Save updated metrics
all_metrics_df.to_csv("ensemble_analysis/all_ensemble_metrics.csv", index=False)

In [ ]:
# 3. Confidence-based Weighted Ensemble
def confidence_weighted_ensemble(probs1, probs2):
    """
    Weight each prediction by its confidence - how far it is from 0.5
    """
    # Calculate confidence for each prediction
    conf1 = abs(probs1 - 0.5) * 2  # Scale to 0-1
    conf2 = abs(probs2 - 0.5) * 2  # Scale to 0-1
    
    # Normalize confidences
    sum_conf = conf1 + conf2
    w1 = conf1 / sum_conf
    w2 = conf2 / sum_conf
    
    # Handle edge case where both confidences are 0
    w1 = np.nan_to_num(w1, nan=0.5)
    w2 = np.nan_to_num(w2, nan=0.5)
    
    # Weighted average of probabilities
    return w1 * probs1 + w2 * probs2

# Create confidence-weighted ensemble predictions
conf_weighted_probs = confidence_weighted_ensemble(transformer_probs, xgb_probs)
conf_weighted_preds = (conf_weighted_probs > 0.5).astype(int)

# Evaluate
metrics_conf_weighted = evaluate_model(
    true_labels, conf_weighted_preds, conf_weighted_probs, 'Confidence-Weighted Ensemble'
)

# Add to metrics dataframe
all_metrics_df = pd.concat([
    all_metrics_df,
    pd.DataFrame([metrics_conf_weighted])
])

print("\nAll models performance including confidence-weighted ensemble:")
display(all_metrics_df)

# Save final metrics
all_metrics_df.to_csv("ensemble_analysis/final_ensemble_metrics.csv", index=False)

In [ ]:
# Create ROC curves for all models
plt.figure(figsize=(12, 8))

# Base models
fpr_transformer, tpr_transformer, _ = roc_curve(true_labels, transformer_probs)
roc_auc_transformer = auc(fpr_transformer, tpr_transformer)
plt.plot(fpr_transformer, tpr_transformer, 
         label=f'Transformer (AUC = {roc_auc_transformer:.4f})')

fpr_xgb, tpr_xgb, _ = roc_curve(true_labels, xgb_probs)
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)
plt.plot(fpr_xgb, tpr_xgb, 
         label=f'XGBoost (AUC = {roc_auc_xgb:.4f})')

# Ensembles
fpr_w_acc, tpr_w_acc, _ = roc_curve(true_labels, weighted_probs_acc)
roc_auc_w_acc = auc(fpr_w_acc, tpr_w_acc)
plt.plot(fpr_w_acc, tpr_w_acc, 
         label=f'Weighted (Acc) (AUC = {roc_auc_w_acc:.4f})')

fpr_w_f1, tpr_w_f1, _ = roc_curve(true_labels, weighted_probs_f1)
roc_auc_w_f1 = auc(fpr_w_f1, tpr_w_f1)
plt.plot(fpr_w_f1, tpr_w_f1, 
         label=f'Weighted (F1) (AUC = {roc_auc_w_f1:.4f})')

fpr_conf, tpr_conf, _ = roc_curve(true_labels, conf_weighted_probs)
roc_auc_conf = auc(fpr_conf, tpr_conf)
plt.plot(fpr_conf, tpr_conf, 
         label=f'Conf-Weighted (AUC = {roc_auc_conf:.4f})')

# Stacking ensembles
fpr_lr, tpr_lr, _ = roc_curve(true_labels, y_prob_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)
plt.plot(fpr_lr, tpr_lr, 
         label=f'Stacking LR (AUC = {roc_auc_lr:.4f})')

fpr_mlp, tpr_mlp, _ = roc_curve(true_labels, y_prob_mlp)
roc_auc_mlp = auc(fpr_mlp, tpr_mlp)
plt.plot(fpr_mlp, tpr_mlp, 
         label=f'Stacking MLP (AUC = {roc_auc_mlp:.4f})')

# Add diagonal line
plt.plot([0, 1], [0, 1], 'k--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for All Models')
plt.legend(loc="lower right")
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('ensemble_analysis/figures/all_models_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# Create Precision-Recall curves for all models
plt.figure(figsize=(12, 8))

# Base models
precision_transformer, recall_transformer, _ = precision_recall_curve(true_labels, transformer_probs)
pr_auc_transformer = auc(recall_transformer, precision_transformer)
plt.plot(recall_transformer, precision_transformer, 
         label=f'Transformer (AUC = {pr_auc_transformer:.4f})')

precision_xgb, recall_xgb, _ = precision_recall_curve(true_labels, xgb_probs)
pr_auc_xgb = auc(recall_xgb, precision_xgb)
plt.plot(recall_xgb, precision_xgb, 
         label=f'XGBoost (AUC = {pr_auc_xgb:.4f})')

# Ensembles
precision_w_acc, recall_w_acc, _ = precision_recall_curve(true_labels, weighted_probs_acc)
pr_auc_w_acc = auc(recall_w_acc, precision_w_acc)
plt.plot(recall_w_acc, precision_w_acc, 
         label=f'Weighted (Acc) (AUC = {pr_auc_w_acc:.4f})')

precision_w_f1, recall_w_f1, _ = precision_recall_curve(true_labels, weighted_probs_f1)
pr_auc_w_f1 = auc(recall_w_f1, precision_w_f1)
plt.plot(recall_w_f1, precision_w_f1, 
         label=f'Weighted (F1) (AUC = {pr_auc_w_f1:.4f})')

precision_conf, recall_conf, _ = precision_recall_curve(true_labels, conf_weighted_probs)
pr_auc_conf = auc(recall_conf, precision_conf)
plt.plot(recall_conf, precision_conf, 
         label=f'Conf-Weighted (AUC = {pr_auc_conf:.4f})')

# Stacking ensembles
precision_lr, recall_lr, _ = precision_recall_curve(true_labels, y_prob_lr)
pr_auc_lr = auc(recall_lr, precision_lr)
plt.plot(recall_lr, precision_lr, 
         label=f'Stacking LR (AUC = {pr_auc_lr:.4f})')

precision_mlp, recall_mlp, _ = precision_recall_curve(true_labels, y_prob_mlp)
pr_auc_mlp = auc(recall_mlp, precision_mlp)
plt.plot(recall_mlp, precision_mlp, 
         label=f'Stacking MLP (AUC = {pr_auc_mlp:.4f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves for All Models')
plt.legend(loc="lower left")
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('ensemble_analysis/figures/all_models_pr_curves.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create confusion matrices for ensemble methods
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Weighted (Accuracy) Ensemble
cm_w_acc = plot_confusion_matrix(
    true_labels, weighted_preds_acc, 'Weighted Ensemble (Accuracy)', axes[0, 0]
)

# Weighted (F1) Ensemble
cm_w_f1 = plot_confusion_matrix(
    true_labels, weighted_preds_f1, 'Weighted Ensemble (F1)', axes[0, 1]
)

# Confidence-Weighted Ensemble
cm_conf = plot_confusion_matrix(
    true_labels, conf_weighted_preds, 'Confidence-Weighted Ensemble', axes[1, 0]
)

# Stacking LR Ensemble
cm_lr = plot_confusion_matrix(
    true_labels, y_pred_lr, 'Stacking LR Ensemble', axes[1, 1]
)

plt.tight_layout()
plt.savefig('ensemble_analysis/figures/ensemble_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Optional: Feature-level Fusion - if time permits and if we have access to features
try:
    # Try to load extracted features
    xgb_features = pd.read_csv("split_data/test_data.csv")
    
    # Check if we have any features
    id_cols = ['Header', 'Position', 'target']
    feature_cols = [col for col in xgb_features.columns if col not in id_cols]
    
    if len(feature_cols) > 0:
        print(f"\nFound {len(feature_cols)} XGBoost features")
        
        # Merge features with transformer probabilities
        fusion_df = pd.merge(
            xgb_features,
            merged_preds[['header', 'position', transformer_prob_col]],
            left_on=['Header', 'Position'],
            right_on=['header', 'position'],
            how='inner'
        )
        
        print(f"Feature fusion dataset size: {fusion_df.shape}")
        
        # Prepare data for feature fusion
        feature_fusion_cols = feature_cols + [transformer_prob_col]
        X_fusion = fusion_df[feature_fusion_cols]
        y_fusion = fusion_df['target']
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_fusion, y_fusion, test_size=0.3, random_state=42, stratify=y_fusion
        )
        
        # Train a model on the combined features
        from sklearn.ensemble import RandomForestClassifier
        
        # Train Random Forest
        rf_fusion = RandomForestClassifier(
            n_estimators=100, 
            max_depth=10, 
            random_state=42,
            n_jobs=-1
        )
        rf_fusion.fit(X_train, y_train)
        
        # Make predictions
        y_test_pred_rf = rf_fusion.predict(X_test)
        y_test_prob_rf = rf_fusion.predict_proba(X_test)[:, 1]
        
        # Evaluate
        metrics_rf_fusion = evaluate_model(
            y_test, y_test_pred_rf, y_test_prob_rf, 'Feature Fusion (RF)'
        )
        
        # Apply to full dataset
        y_pred_rf = rf_fusion.predict(X_fusion)
        y_prob_rf = rf_fusion.predict_proba(X_fusion)[:, 1]
        
        # Evaluate on full dataset
        metrics_rf_full = evaluate_model(
            y_fusion, y_pred_rf, y_prob_rf, 'Feature Fusion (RF) Full'
        )
        
        # Add to metrics dataframe
        all_metrics_df = pd.concat([
            all_metrics_df,
            pd.DataFrame([metrics_rf_full])
        ])
        
        print("\nAll models including feature fusion:")
        display(all_metrics_df)
        
        # Save updated metrics
        all_metrics_df.to_csv("ensemble_analysis/all_metrics_with_fusion.csv", index=False)
        
        # Feature importance analysis
        if hasattr(rf_fusion, 'feature_importances_'):
            importance = rf_fusion.feature_importances_
            indices = np.argsort(importance)[::-1]
            
            # Plot feature importance
            plt.figure(figsize=(12, 8))
            
            # Show top 30 features
            top_n = min(30, len(feature_fusion_cols))
            plt.bar(range(top_n), importance[indices[:top_n]])
            plt.xticks(
                range(top_n), 
                [feature_fusion_cols[i] for i in indices[:top_n]], 
                rotation=90
            )
            plt.xlabel('Feature')
            plt.ylabel('Importance')
            plt.title('Feature Importance in Fusion Model')
            plt.tight_layout()
            plt.savefig('ensemble_analysis/figures/fusion_feature_importance.png', dpi=300, bbox_inches='tight')
            plt.show()
            
            # Save top features
            top_features = pd.DataFrame({
                'Feature': [feature_fusion_cols[i] for i in indices],
                'Importance': importance[indices]
            })
            top_features.to_csv('ensemble_analysis/fusion_feature_importance.csv', index=False)
            
            print("\nTop 10 features in fusion model:")
            display(top_features.head(10))
            
    else:
        print("No features found in XGBoost test data")
        
except Exception as e:
    print(f"\nCould not implement feature-level fusion: {e}")

In [ ]:
# Create a visual summary of the performance of all models
plt.figure(figsize=(14, 10))

# Sort models by F1 score
sorted_metrics = all_metrics_df.sort_values('F1 Score', ascending=False).reset_index(drop=True)

# Plot metrics for each model
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# Plot metrics as grouped bars
bar_width = 0.15
index = np.arange(len(sorted_metrics))

for i, metric in enumerate(metrics_to_plot):
    plt.bar(
        index + i*bar_width, 
        sorted_metrics[metric], 
        bar_width,
        label=metric,
        color=colors[i]
    )

plt.xlabel('Model')
plt.ylabel('Score')
plt.title('Performance Comparison of All Models')
plt.xticks(index + bar_width*2, sorted_metrics['Model'], rotation=45, ha='right')
plt.legend(loc='lower right')
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('ensemble_analysis/figures/all_models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Create a summary table of relative improvement
base_model = 'XGBoost' if metrics_xgb['F1 Score'] > metrics_transformer['F1 Score'] else 'Transformer'
base_f1 = metrics_xgb['F1 Score'] if base_model == 'XGBoost' else metrics_transformer['F1 Score']

# Calculate improvements
improvement_df = all_metrics_df.copy()
improvement_df['F1 Improvement'] = improvement_df['F1 Score'] - base_f1
improvement_df['F1 Improvement %'] = (improvement_df['F1 Score'] - base_f1) / base_f1 * 100
improvement_df = improvement_df.sort_values('F1 Improvement', ascending=False)

print(f"\nImprovement over best base model ({base_model}, F1={base_f1:.4f}):")
display(improvement_df[['Model', 'F1 Score', 'F1 Improvement', 'F1 Improvement %']])

# Save improvement summary
improvement_df.to_csv('ensemble_analysis/improvement_summary.csv', index=False)

# Create a final textual summary
best_model = improvement_df.iloc[0]['Model']
best_f1 = improvement_df.iloc[0]['F1 Score']
best_improvement = improvement_df.iloc[0]['F1 Improvement %']

summary = f"""
# Phosphorylation Site Prediction Ensemble Analysis Summary

## Base Models
- Transformer Model: {metrics_transformer['Accuracy']:.4f} Accuracy, {metrics_transformer['F1 Score']:.4f} F1 Score
- XGBoost Model: {metrics_xgb['Accuracy']:.4f} Accuracy, {metrics_xgb['F1 Score']:.4f} F1 Score

## Best Ensemble Method
- Model: {best_model}
- F1 Score: {best_f1:.4f}
- Improvement over best base model: {best_improvement:.2f}%

## Ensemble Methods Analysis
1. **Stacking Ensembles**: Trained a meta-model (Logistic Regression or MLP) using base model predictions as features
2. **Weighted Ensembles**: Combined predictions with optimized weights for accuracy or F1 score
3. **Confidence-Weighted Ensemble**: Dynamically weighted predictions based on confidence

## Key Findings
- The ensembles successfully combined the strengths of both models
- Model agreement analysis showed complementary prediction patterns
- Sequence context analysis revealed patterns where models succeed or fail
- The optimal weight for the transformer model was {weight_f1:.4f} when optimizing for F1 score

## Recommendations
- Use {best_model} for best overall performance
- Consider applying different ensemble strategies for different types of sequences
- Further explore the sequence patterns where both models fail to improve future models
"""

# Save summary
with open('ensemble_analysis/summary.md', 'w') as f:
    f.write(summary)

print("\nAnalysis complete. Summary saved to ensemble_analysis/summary.md")
print(summary)